<a href="https://colab.research.google.com/github/RuthKassahun/Skin-Lesion-Classification/blob/main/Testing_Multi_Class_GLCM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GLCM Multi class

> Indented block



## Import Libraries

In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import os
import seaborn as sns
import pandas as pd
from skimage.filters import sobel
from skimage.feature import greycomatrix, greycoprops
from skimage.measure import shannon_entropy
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn import metrics
from pathlib import Path
import xgboost as xgb
import lightgbm as lgb
from PIL import Image, ImageOps
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, auc, roc_curve, roc_auc_score, recall_score, confusion_matrix


## Load Data

In [24]:
SIZE = 224

In [25]:
def feature_extractor(dataset):
  image_dataset  = pd.DataFrame()
  distance = [11, 21]
  angle = [0, np.pi/4, np.pi/2, 3*np.pi/4]
  for image in range(dataset.shape[0]):
    img = dataset[image]
    df = pd.DataFrame()
    count = 1
    for dist in distance:
      for ang in angle:
        GLCM = greycomatrix(img, [dist], [ang])      
        GLCM_Energy = greycoprops(GLCM, 'energy')[0]
        df[f'Energy{count}'] = GLCM_Energy
        GLCM_corr = greycoprops(GLCM, 'correlation')[0]
        df[f'Corr{count}'] = GLCM_corr             
        GLCM_hom = greycoprops(GLCM, 'homogeneity')[0]
        df[f'Homogen{count}'] = GLCM_hom       
        GLCM_contr = greycoprops(GLCM, 'contrast')[0]
        df[f'Contrast{count}'] = GLCM_contr
        count = count + 1
    image_dataset = image_dataset.append(df)    
  return image_dataset

In [26]:
train_images = []
train_labels = [] 
hu_features = []
#for directory_path in glob.glob("cell_images/train/*"):
for directory_path in glob.glob("/content/drive/MyDrive/Multiclass/multi_train/*"):
    label = directory_path.split("/")[-1]
    #print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, 0) #Reading color images
        img = cv2.resize(img, (SIZE, SIZE)) #Resize images
        train_images.append(img)
        train_labels.append(label)
        
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [27]:
print(train_labels)

['scc' 'scc' 'scc' ... 'bcc' 'bcc' 'bcc']


In [1]:
from zipfile import ZipFile
file_name = '/content/sample_check.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [75]:
test_images = []
test_ID_2 = []
counter = 1
#hu_features_test = []
test_array=[]
#for directory_path in glob.glob("cell_images/test/*"): 
for directory_path in glob.glob("/content/sample_check/*"):
   test_label = directory_path.split("/")[-1]
   print(test_label)
   test_ID_2.append(test_label)
   test_array.append(counter)
   counter = counter+1
   for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, 0)
        #img = cv2.resize(img, (SIZE, SIZE))
        test_images.append(img)
        #test_ID_1.append(test_label)
        
test_images = np.array(test_images)
#test_ID = np.array(test_ID)
test_array = np.array(test_array)


nev00003.jpg
nev00001.jpg
nev00004.jpg
nev00045.jpg
nev00002.jpg
nev00031.jpg
nev00044.jpg


In [16]:
print(test_ID_2)

['nev00003.jpg', 'nev00001.jpg', 'nev00004.jpg', 'nev00045.jpg', 'nev00002.jpg', 'nev00031.jpg', 'nev00044.jpg']


In [20]:
print(test_array)

[1 2 3 4 5 6 7]


In [21]:
res = pd.DataFrame(test_array)
res.index = test_ID_2 # its important for comparison
res.columns = ["prediction"]
res.to_csv("prediction_results.csv")

In [ ]:
test_image = []
test_image_ID= []
#hu_features_test = []
for directory_path in glob.glob("/content/drive/MyDrive/Multiclass/multi_test/*"):
    test_label = directory_path.split("/")[-1]
    print(test_label)
    test_image_ID.append(test_label)
for directory_path in glob.glob("/content/drive/MyDrive/Multiclass/multi_test"):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img1 = cv2.imread(img_path, 0)
        img1 = cv2.resize(img1, (SIZE, SIZE))
        test_image.append(img1)
        
test_image = np.array(test_image)



## Label Encoding

In [93]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
#le.fit(test_labels)
#test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [95]:
x_train, y_train, x_test = train_images, train_labels_encoded, test_image

In [96]:
image_features = feature_extractor(x_train)
X_for_ML =image_features

In [114]:
image_features = feature_extractor(x_test)
X_test_ML =image_features

In [115]:
x_train = X_for_ML
x_test = X_test_ML


In [116]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [117]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [118]:
over=RandomOverSampler(random_state=0,sampling_strategy='minority')
X_res,Y_res=over.fit_resample(x_train,y_train)

In [119]:
print(sum(Y_res==0))

1993


In [120]:
over=RandomOverSampler(random_state=0,sampling_strategy='minority')
X_result,Y_result=over.fit_resample(X_res,Y_res)

In [121]:
print(sum(Y_result==0))

2713


## Random Forest

In [122]:
pipe2 = Pipeline([
                 ("scale", StandardScaler()),
                 ("classifier", RandomForestClassifier(n_estimators = 100, random_state = 42))
                 ])

In [123]:
pipe2.fit(X_result,Y_result)

Pipeline(steps=[('scale', StandardScaler()),
                ('classifier', RandomForestClassifier(random_state=42))])

In [124]:
# Test set evaluation
prediction = pipe2.predict(x_test)


In [125]:
res = pd.DataFrame(prediction)
res.index = test_image_ID # its important for comparison
res.columns = ["prediction"]
res.to_csv("prediction_multiclass.csv")

